In [1]:
import tensorflow as tf
tf.__version__
tf.keras.backend.clear_session()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
# import keras.backend as K
# from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.regularizers import l1,l2, l1_l2
# from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Layer, Permute, multiply
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv1D, Dropout, Dense, BatchNormalization, AveragePooling1D, ReLU, LayerNormalization
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Lambda, Reshape, Flatten, SeparableConv1D, Activation, GlobalAveragePooling1D
from tensorflow.keras.layers import Input, Add, MaxPooling1D, MultiHeadAttention, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.compat.v1.keras.models import load_model
# from tensorflow.compat.v1.keras.layers import CuDNNGRU
import sklearn
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight 

In [3]:
# data_b2= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B2_final.csv')
# data_b3= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B3_final.csv')
# data_b4= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B4_final.csv')
# data_b5= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B5_final.csv')
# data_b6= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B6_final.csv')
# data_b7= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B7_final.csv')
# data_b8= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B8_final.csv')
# data_b8A= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B8A_final.csv')
# data_b11= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B11_final.csv')
# data_b12= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Sentinel/Refined_Data/B12_final.csv')

data_cbi= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/CBI_final.csv')
data_bsi= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/BSI_final.csv')
data_evi2= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/EVI2_final.csv')
data_mndwi= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/MNDWI_final.csv')
data_msavi= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/MSAVI_final.csv')
data_nbr= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/NBR_final.csv')
data_ndvi= pd.read_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Refined_Data/NDVI_final.csv')

In [4]:
#interpolate NaN for each row
# data_b2 = data_b2.apply(pd.to_numeric, errors='coerce')
# data_b3 = data_b3.apply(pd.to_numeric, errors='coerce')
# data_b4 = data_b4.apply(pd.to_numeric, errors='coerce')
# data_b5 = data_b5.apply(pd.to_numeric, errors='coerce')
# data_b6 = data_b6.apply(pd.to_numeric, errors='coerce')
# data_b7 = data_b7.apply(pd.to_numeric, errors='coerce')
# data_b8 = data_b8.apply(pd.to_numeric, errors='coerce')
# data_b8A = data_b8A.apply(pd.to_numeric, errors='coerce')
# data_b11 = data_b11.apply(pd.to_numeric, errors='coerce')
# data_b12 = data_b12.apply(pd.to_numeric, errors='coerce')

data_cbi_ = data_cbi.values
data_bsi_ = data_bsi.values
data_evi2_ = data_evi2.values
data_mndwi_ = data_mndwi.values
data_msavi_ = data_msavi.values
data_nbr_ = data_nbr.values
data_ndvi_ = data_ndvi.values

cbi_pd = pd.DataFrame(data_cbi_)
bsi_pd = pd.DataFrame(data_bsi_)
evi2_pd = pd.DataFrame(data_evi2_)
mndwi_pd = pd.DataFrame(data_mndwi_)
msavi_pd = pd.DataFrame(data_msavi_)
nbr_pd = pd.DataFrame(data_nbr_)
ndvi_pd = pd.DataFrame(data_ndvi_)

cbi_pd = cbi_pd.iloc[:,3:42].astype(float)
bsi_pd = bsi_pd.iloc[:,3:42].astype(float)
evi2_pd  = evi2_pd.iloc[:,3:42].astype(float)
mndwi_pd  = mndwi_pd.iloc[:,3:42].astype(float)
msavi_pd = msavi_pd.iloc[:,3:42].astype(float)
nbr_pd  = nbr_pd.iloc[:,3:42].astype(float)
ndvi_pd  = ndvi_pd.iloc[:,3:42].astype(float)

# data_b2_int = data_b2.interpolate(method='linear', limit_direction='both', axis=1)
# data_b3_int = data_b3.interpolate(method='linear', limit_direction='both', axis=1)
# data_b4_int = data_b4.interpolate(method='linear', limit_direction='both', axis=1)
# data_b5_int = data_b5.interpolate(method='linear', limit_direction='both', axis=1)
# data_b6_int = data_b6.interpolate(method='linear', limit_direction='both', axis=1)
# data_b7_int = data_b7.interpolate(method='linear', limit_direction='both', axis=1)
# data_b8_int = data_b8.interpolate(method='linear', limit_direction='both', axis=1)
# data_b8A_int = data_b8A.interpolate(method='linear', limit_direction='both', axis=1)
# data_b11_int = data_b11.interpolate(method='linear', limit_direction='both', axis=1)
# data_b12_int = data_b12.interpolate(method='linear', limit_direction='both', axis=1)

data_cbi_int = cbi_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
data_bsi_int = bsi_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
data_evi2_int = evi2_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
data_mndwi_int = mndwi_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
data_msavi_int = msavi_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
data_nbr_int = nbr_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
data_ndvi_int = ndvi_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)

# print('Number of nan for B2 after:', data_b2_int.isnull().sum().sum())
# print('Number of nan for B3 after:', data_b3_int.isnull().sum().sum())
# print('Number of nan for B4 after:', data_b4_int.isnull().sum().sum())
# print('Number of nan for B5 after:', data_b5_int.isnull().sum().sum())
# print('Number of nan for B6 after:', data_b6_int.isnull().sum().sum())
# print('Number of nan for B7 after:', data_b7_int.isnull().sum().sum())
# print('Number of nan for B8 after:', data_b8_int.isnull().sum().sum())
# print('Number of nan for B8A after:', data_b8A_int.isnull().sum().sum())
# print('Number of nan for B11 after:', data_b11_int.isnull().sum().sum())
# print('Number of nan for B12 after:', data_b12_int.isnull().sum().sum())

print('Number of nan for CBI after:', np.count_nonzero(np.isnan(data_cbi_int)))
print('Number of nan for BSI after:', np.count_nonzero(np.isnan(data_bsi_int)))
print('Number of nan for EVI2 after:', np.count_nonzero(np.isnan(data_evi2_int)))
print('Number of nan for MNDWI after:', data_mndwi_int.isnull().sum().sum())
print('Number of nan for MSAVI after:', data_msavi_int.isnull().sum().sum())
print('Number of nan for NBR after:', data_nbr_int.isnull().sum().sum())
print('Number of nan for NDVI after:', data_ndvi_int.isnull().sum().sum())

Number of nan for CBI after: 0
Number of nan for BSI after: 0
Number of nan for EVI2 after: 0
Number of nan for MNDWI after: 0
Number of nan for MSAVI after: 0
Number of nan for NBR after: 0
Number of nan for NDVI after: 0


In [5]:
# print(data_cbi_int.max())
# print(data_bsi_int.max())
# print(data_evi2_int.max())
# print(data_mndwi_int.max())
# print(data_msavi_int.max())
# print(data_nbr_int.max())
# print(data_ndvi_int.max())

In [6]:
comb_all = pd.concat([data_cbi_int, data_bsi_int, data_evi2_int,
                      data_mndwi_int, data_msavi_int, 
                      data_nbr_int, data_ndvi_int], axis =0, ignore_index=True)

In [7]:
comb_all_lc = pd.concat([data_cbi.iloc[:,0:3], data_bsi.iloc[:,0:3], data_evi2.iloc[:,0:3],
                            data_mndwi.iloc[:,0:3], data_msavi.iloc[:,0:3],
                            data_nbr.iloc[:,0:3], data_ndvi.iloc[:,0:3]], axis =0, ignore_index=True)

In [8]:
# print(comb_all_lc)

In [9]:
# print(comb_all_lc.Class.value_counts())

In [10]:
y = data_cbi.iloc[:,2]
y = np.asarray(y).astype(np.float32)
y_cat = to_categorical(y)
y_cat = y_cat[:,1:8] # num_class + 1

In [11]:
# print(y.shape)

In [12]:
# print(y_cat)
# print(y_cat.shape)

In [13]:
# print(np.unique(y))

In [14]:
class_weights = compute_class_weight(class_weight = "balanced",
                                      classes = np.unique(y), y=y)
class_weights = dict(enumerate(class_weights))
# class_weights = dict(zip(np.unique(train_classes), class_weights))
print(class_weights)

{0: 1.2355889724310778, 1: 0.985014985014985, 2: 0.4977284199899041, 3: 2.9969604863221884, 4: 1.5916061339790153, 5: 0.8072042570609906, 6: 1.0357142857142858}


In [15]:
# print(comb_all.shape)
# print(comb_all)

In [16]:
# comb_b2 = comb_all.iloc[0:1939:,3:31]
# comb_b3 = comb_all.iloc[1939:3878:,3:31]
# comb_b4 = comb_all.iloc[3878:5817:,3:31]
# comb_b5 = comb_all.iloc[5817:7756:,3:31]
# comb_b6 = comb_all.iloc[7756:9695:,3:31]
# comb_b7 = comb_all.iloc[9695:11634:,3:31]
# comb_b8 = comb_all.iloc[11634:13573:,3:31]
# comb_b8A = comb_all.iloc[13573:15512:,3:31]
# comb_b11 = comb_all.iloc[15512:17451:,3:31]
# comb_b12 = comb_all.iloc[17451:19390:,3:31]

comb_cbi = comb_all.iloc[0:1972:,:]
comb_bsi = comb_all.iloc[1972:3944:,:]
comb_evi2 = comb_all.iloc[3944:5916:,:]
comb_mndwi = comb_all.iloc[5916:7888:,:]
comb_msavi = comb_all.iloc[7888:9860:,:]
comb_nbr = comb_all.iloc[9860:11832:,:]
comb_ndvi = comb_all.iloc[11832:13804:,:]

# comb_b2= np.asarray(comb_b2).astype(np.float32)
# comb_b3= np.asarray(comb_b3).astype(np.float32)
# comb_b4= np.asarray(comb_b4).astype(np.float32)
# comb_b5= np.asarray(comb_b5).astype(np.float32)
# comb_b6= np.asarray(comb_b6).astype(np.float32)
# comb_b7= np.asarray(comb_b7).astype(np.float32)
# comb_b8= np.asarray(comb_b8).astype(np.float32)
# comb_b8A= np.asarray(comb_b8A).astype(np.float32)
# comb_b11= np.asarray(comb_b11).astype(np.float32)
# comb_b12= np.asarray(comb_b12).astype(np.float32)

comb_cbi= np.asarray(comb_cbi).astype(np.float64)
comb_bsi= np.asarray(comb_bsi).astype(np.float64)
comb_evi2= np.asarray(comb_evi2).astype(np.float64)
comb_mndwi= np.asarray(comb_mndwi).astype(np.float64)
comb_msavi= np.asarray(comb_msavi).astype(np.float64)
comb_nbr= np.asarray(comb_nbr).astype(np.float64)
comb_ndvi= np.asarray(comb_ndvi).astype(np.float64)

In [17]:
comb_stack = np.stack((comb_cbi, comb_bsi, comb_evi2,
                       comb_mndwi, comb_msavi, comb_nbr, 
                       comb_ndvi), axis=1)

# comb_stack = np.stack((comb_cbi, comb_bsi, comb_evi2), axis=1)
# print(comb_stack.shape)

comb_stack_reshape = comb_stack.reshape(comb_stack.shape[0], comb_stack.shape[2], comb_stack.shape[1])
print(comb_stack_reshape.shape)

X_train, X_test, y_train, y_test = train_test_split(comb_stack_reshape , y_cat, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1972, 39, 7)
(1380, 39, 7) (1380, 7) (592, 39, 7) (592, 7)


In [18]:
# print(comb_cbi.shape)

In [19]:
# stack_ver = np.hstack((vert_cbi, vert_bsi, vert_evi2,
#                        vert_mndwi, vert_msavi, vert_nbr, 
#                        vert_ndvi))
# print(stack_ver.shape)

In [20]:
# import seaborn as sns 

# x = ['CBI','BSI','EVI2', 'MNDWI', 'MSAVI', 'NBR', 'NDVI']
# # enumerate label
# for i in enumerate(x):
#     print(i)

# # plot correlation matrix
# rf_pd = pd.DataFrame(data=stack_ver, columns=list(x))
# matrix = np.triu(rf_pd.corr())
# plt.figure(figsize=(12,8))
# sns.heatmap(rf_pd.corr(), annot=True, mask=matrix)

In [21]:
num_classes = len(np.unique(y))
input_shape=(X_train.shape[1], X_train.shape[2])
print(num_classes)
print(input_shape)

7
(39, 7)


In [22]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

tf.keras.utils.get_custom_objects().update({'swish': swish})

In [23]:
def ReshapeLayer(x):
    shape = x.shape
    # 1 possibility: H,W*channel
    reshape = Reshape((shape[1],shape[2]))(x)
    # 2 possibility: W,H*channel
    # transpose = Permute((2,1,3))(x)
    # reshape = Reshape((shape[1],shape[2]*shape[3]))(transpose)
    return reshape

In [24]:
def attention_block(inputs, output_shape):
    a = Permute((2, 1))(inputs)
    a = Dense(output_shape, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [25]:
import tensorflow as tf
from tensorflow.keras.metrics import Metric

class CustomMeanIoU(Metric):
    def __init__(self, num_classes, name='custom_mean_iou', **kwargs):
        super(CustomMeanIoU, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.confusion_matrix = self.add_weight(
            'confusion_matrix',
            shape=(num_classes, num_classes),
            initializer='zeros',
            dtype=tf.float32)
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # First, ensure y_pred is class indices (argmax) and cast to int32
        if len(tf.shape(y_pred)) > 1 and tf.shape(y_pred)[-1] > 1:
            y_pred = tf.cast(tf.argmax(y_pred, axis=-1), tf.int32)
        else:
            # If not one-hot, ensure it's still int32
            y_pred = tf.cast(y_pred, tf.int32)
        
        # Then ensure y_true is the same format (class indices as int32)
        if len(tf.shape(y_true)) > 1 and tf.shape(y_true)[-1] > 1:
            y_true = tf.cast(tf.argmax(y_true, axis=-1), tf.int32)
        else:
            # If not one-hot, ensure it's still int32
            y_true = tf.cast(y_true, tf.int32)
            
        # Convert to 1D tensors
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])
        
        # Calculate confusion matrix
        cm_update = tf.math.confusion_matrix(
            y_true, y_pred, self.num_classes, 
            dtype=tf.float32)
        
        # Update the total confusion matrix
        self.confusion_matrix.assign_add(cm_update)
        
    def result(self):
        cm = self.confusion_matrix
        
        # Extract per-class TP, FP, FN
        tp = tf.linalg.diag_part(cm)
        sum_over_row = tf.reduce_sum(cm, axis=1)
        sum_over_col = tf.reduce_sum(cm, axis=0)
        fp = sum_over_col - tp
        fn = sum_over_row - tp
        
        # Calculate IoU per class
        denominator = tp + fp + fn
        
        # Avoid division by zero
        iou = tf.math.divide_no_nan(tp, denominator)
        
        # Calculate mean IoU (average over classes)
        valid_classes = tf.cast(denominator > 0, tf.float32)
        num_valid = tf.reduce_sum(valid_classes)
        
        # Return mean IoU
        return tf.math.divide_no_nan(tf.reduce_sum(iou), num_valid)
        
    def reset_state(self):
        # Reset the confusion matrix at the start of each epoch
        self.confusion_matrix.assign(tf.zeros((self.num_classes, self.num_classes)))

    def get_config(self):
        config = super(CustomMeanIoU, self).get_config()
        config.update({"num_classes": self.num_classes})
        return config

In [26]:
%%time
#Best set up

def make_model(input_shape):
    input_layer = Input(input_shape)
    
    xfin = Lambda(ReshapeLayer)(input_layer)
    xfin = Bidirectional(GRU(100, activation='tanh', return_sequences=True))(xfin)
    xfin = Dropout(0.5)(xfin)
    xfin = Bidirectional(GRU(100, activation='tanh', return_sequences=False))(xfin)
    xfin = Dropout(0.5)(xfin)
    xfin = Dense(64, activation='relu')(xfin)

    output_layer = Dense(num_classes, activation="softmax")(xfin)

    return Model(inputs=input_layer, outputs=output_layer)

model = make_model(input_shape= input_shape)
model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=optimizers.AdamW(learning_rate=0.001), 
                    metrics=[CustomMeanIoU(num_classes=7)])

checkpoint = ModelCheckpoint(filepath='D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Saved_Model/LC_DAS_Citarum_exp03_dualBiLSTM_mIoU.h5',
                              monitor='val_custom_mean_iou',save_best_only=True,verbose=1,mode='max')
early_stop = EarlyStopping(monitor='val_custom_mean_iou',patience=100,restore_best_weights=True,mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_custom_mean_iou', factor=0.1,patience=30, min_lr=0.00001)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size =20,
                    epochs=500, callbacks = [checkpoint, early_stop, reduce_lr], 
                    class_weight=class_weights)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 39, 7)]           0         
                                                                 
 lambda (Lambda)             (None, 39, 7)             0         
                                                                 
 bidirectional (Bidirection  (None, 39, 200)           65400     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 39, 200)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               181200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 200)               0     

D:\anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


69/69 [==============================] - ETA: 0s - loss: 1.0480 - custom_mean_iou: 0.4294
Epoch 2: val_custom_mean_iou improved from 0.39411 to 0.46013, saving model to D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Saved_Model\LC_DAS_Citarum_exp03_dualBiLSTM_mIoU.h5
69/69 [==============================] - 4s 57ms/step - loss: 1.0480 - custom_mean_iou: 0.4294 - val_loss: 0.9101 - val_custom_mean_iou: 0.4601 - lr: 0.0010
Epoch 3/500
69/69 [==============================] - ETA: 0s - loss: 0.9533 - custom_mean_iou: 0.4747
Epoch 3: val_custom_mean_iou improved from 0.46013 to 0.50093, saving model to D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Saved_Model\LC_DAS_Citarum_exp03_dualBiLSTM_mIoU.h5
69/69 [==============================] - 4s 62ms/step - loss: 0.9533 - custom_mean_iou: 0.4747 - val_loss: 0.8189 - val_custom_mean_iou: 0.5009 - lr: 0.0010
Epoch 4/500
68/69 [============================>.] - ETA: 0s - loss: 0.8993 - custom_mean_iou: 0.5001
Epoch 4: v

Epoch 24/500
69/69 [==============================] - ETA: 0s - loss: 0.4321 - custom_mean_iou: 0.7029
Epoch 24: val_custom_mean_iou improved from 0.73540 to 0.75641, saving model to D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Saved_Model\LC_DAS_Citarum_exp03_dualBiLSTM_mIoU.h5
69/69 [==============================] - 4s 58ms/step - loss: 0.4321 - custom_mean_iou: 0.7029 - val_loss: 0.3684 - val_custom_mean_iou: 0.7564 - lr: 0.0010
Epoch 25/500
69/69 [==============================] - ETA: 0s - loss: 0.4097 - custom_mean_iou: 0.7198
Epoch 25: val_custom_mean_iou did not improve from 0.75641
69/69 [==============================] - 4s 52ms/step - loss: 0.4097 - custom_mean_iou: 0.7198 - val_loss: 0.4229 - val_custom_mean_iou: 0.6965 - lr: 0.0010
Epoch 26/500
68/69 [============================>.] - ETA: 0s - loss: 0.3882 - custom_mean_iou: 0.7294
Epoch 26: val_custom_mean_iou did not improve from 0.75641
69/69 [==============================] - 4s 53ms/step - loss: 0.389

Epoch 47/500
68/69 [============================>.] - ETA: 0s - loss: 0.2457 - custom_mean_iou: 0.8244
Epoch 47: val_custom_mean_iou did not improve from 0.79149
69/69 [==============================] - 4s 53ms/step - loss: 0.2444 - custom_mean_iou: 0.8264 - val_loss: 0.3295 - val_custom_mean_iou: 0.7848 - lr: 1.0000e-04
Epoch 48/500
69/69 [==============================] - ETA: 0s - loss: 0.2411 - custom_mean_iou: 0.8255
Epoch 48: val_custom_mean_iou did not improve from 0.79149
69/69 [==============================] - 4s 53ms/step - loss: 0.2411 - custom_mean_iou: 0.8255 - val_loss: 0.3281 - val_custom_mean_iou: 0.7684 - lr: 1.0000e-04
Epoch 49/500
69/69 [==============================] - ETA: 0s - loss: 0.2558 - custom_mean_iou: 0.8121
Epoch 49: val_custom_mean_iou did not improve from 0.79149
69/69 [==============================] - 4s 54ms/step - loss: 0.2558 - custom_mean_iou: 0.8121 - val_loss: 0.3284 - val_custom_mean_iou: 0.7721 - lr: 1.0000e-04
Epoch 50/500
69/69 [===========

...

69/69 [==============================] - ETA: 0s - loss: 0.2011 - custom_mean_iou: 0.8400
Epoch 123: val_custom_mean_iou did not improve from 0.79149
69/69 [==============================] - 5s 71ms/step - loss: 0.2011 - custom_mean_iou: 0.8400 - val_loss: 0.3203 - val_custom_mean_iou: 0.7830 - lr: 1.0000e-05
Epoch 124/500
69/69 [==============================] - ETA: 0s - loss: 0.2060 - custom_mean_iou: 0.8424
Epoch 124: val_custom_mean_iou did not improve from 0.79149
69/69 [==============================] - 4s 59ms/step - loss: 0.2060 - custom_mean_iou: 0.8424 - val_loss: 0.3220 - val_custom_mean_iou: 0.7895 - lr: 1.0000e-05
Epoch 125/500
68/69 [============================>.] - ETA: 0s - loss: 0.2144 - custom_mean_iou: 0.8438
Epoch 125: val_custom_mean_iou did not improve from 0.79149
69/69 [==============================] - 4s 53ms/step - loss: 0.2135 - custom_mean_iou: 0.8437 - val_loss: 0.3192 - val_custom_mean_iou: 0.7832 - lr: 1.0000e-05
Epoch 126/500
69/69 [==================

Epoch 148/500
68/69 [============================>.] - ETA: 0s - loss: 0.2119 - custom_mean_iou: 0.8469
Epoch 148: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 52ms/step - loss: 0.2128 - custom_mean_iou: 0.8450 - val_loss: 0.3211 - val_custom_mean_iou: 0.7859 - lr: 1.0000e-05
Epoch 149/500
69/69 [==============================] - ETA: 0s - loss: 0.2118 - custom_mean_iou: 0.8356
Epoch 149: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 53ms/step - loss: 0.2118 - custom_mean_iou: 0.8356 - val_loss: 0.3194 - val_custom_mean_iou: 0.7859 - lr: 1.0000e-05
Epoch 150/500
68/69 [============================>.] - ETA: 0s - loss: 0.1969 - custom_mean_iou: 0.8579
Epoch 150: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 53ms/step - loss: 0.2003 - custom_mean_iou: 0.8578 - val_loss: 0.3196 - val_custom_mean_iou: 0.7838 - lr: 1.0000e-05
Epoch 151/500
68/69 [====

68/69 [============================>.] - ETA: 0s - loss: 0.2044 - custom_mean_iou: 0.8489
Epoch 173: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 53ms/step - loss: 0.2089 - custom_mean_iou: 0.8451 - val_loss: 0.3194 - val_custom_mean_iou: 0.7859 - lr: 1.0000e-05
Epoch 174/500
69/69 [==============================] - ETA: 0s - loss: 0.2005 - custom_mean_iou: 0.8527
Epoch 174: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 55ms/step - loss: 0.2005 - custom_mean_iou: 0.8527 - val_loss: 0.3184 - val_custom_mean_iou: 0.7892 - lr: 1.0000e-05
Epoch 175/500
69/69 [==============================] - ETA: 0s - loss: 0.2000 - custom_mean_iou: 0.8539
Epoch 175: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 63ms/step - loss: 0.2000 - custom_mean_iou: 0.8539 - val_loss: 0.3208 - val_custom_mean_iou: 0.7832 - lr: 1.0000e-05
Epoch 176/500
69/69 [==================

69/69 [==============================] - ETA: 0s - loss: 0.2051 - custom_mean_iou: 0.8361
Epoch 198: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 59ms/step - loss: 0.2051 - custom_mean_iou: 0.8361 - val_loss: 0.3195 - val_custom_mean_iou: 0.7832 - lr: 1.0000e-05
Epoch 199/500
69/69 [==============================] - ETA: 0s - loss: 0.2072 - custom_mean_iou: 0.8446
Epoch 199: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 59ms/step - loss: 0.2072 - custom_mean_iou: 0.8446 - val_loss: 0.3176 - val_custom_mean_iou: 0.7859 - lr: 1.0000e-05
Epoch 200/500
69/69 [==============================] - ETA: 0s - loss: 0.1957 - custom_mean_iou: 0.8452
Epoch 200: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 62ms/step - loss: 0.1957 - custom_mean_iou: 0.8452 - val_loss: 0.3207 - val_custom_mean_iou: 0.7832 - lr: 1.0000e-05
Epoch 201/500
69/69 [==================

69/69 [==============================] - ETA: 0s - loss: 0.2001 - custom_mean_iou: 0.8436
Epoch 223: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 57ms/step - loss: 0.2001 - custom_mean_iou: 0.8436 - val_loss: 0.3172 - val_custom_mean_iou: 0.7859 - lr: 1.0000e-05
Epoch 224/500
69/69 [==============================] - ETA: 0s - loss: 0.1884 - custom_mean_iou: 0.8540
Epoch 224: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 57ms/step - loss: 0.1884 - custom_mean_iou: 0.8540 - val_loss: 0.3186 - val_custom_mean_iou: 0.7832 - lr: 1.0000e-05
Epoch 225/500
69/69 [==============================] - ETA: 0s - loss: 0.1939 - custom_mean_iou: 0.8512
Epoch 225: val_custom_mean_iou did not improve from 0.79246
69/69 [==============================] - 4s 59ms/step - loss: 0.1939 - custom_mean_iou: 0.8512 - val_loss: 0.3178 - val_custom_mean_iou: 0.7859 - lr: 1.0000e-05
Epoch 226/500
69/69 [==================

In [ ]:
n_epochs = len(history.history['loss'])

acc = history.history['custom_mean_iou']
val_acc = history.history['val_custom_mean_iou']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(n_epochs) #change it based on epoch needed to finish building the model

plt.figure(figsize=(16, 16))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

labels, counts = np.unique(y, return_counts=True)
labels = labels -1
classes = ['Forest','Plantation','Agriculture', 'Shrubland', 'Bareland', 'Urban', 'Waterbody']

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
score = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred, labels=labels)

# plot the confusion matrix of LGB
dl_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(dl_cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)


# Show all labels
ax.set(xticks=np.arange(dl_cm.shape[1]),
        yticks=np.arange(dl_cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes, yticklabels=classes,
        title='Normalized Confusion Matrix',
        ylabel='True label',
        xlabel='Predicted label')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = '.2f'
thresh = dl_cm.max() / 2.
for i in range(dl_cm.shape[0]):
    for j in range(dl_cm.shape[1]):
        ax.text(j, i, format(dl_cm[i, j], fmt),
                ha="center", va="center",
                color="white" if dl_cm[i, j] > thresh else "black")
fig.tight_layout()

In [ ]:
# Calculate per-class IoU and mean IoU from raw confusion matrix
def calculate_iou_from_cm(cm):
    n_classes = cm.shape[0]
    class_iou = np.zeros(n_classes)
    
    for i in range(n_classes):
        # True positives: diagonal elements
        tp = cm[i, i]
        
        # False positives: sum of column i minus diagonal element
        fp = np.sum(cm[:, i]) - tp
        
        # False negatives: sum of row i minus diagonal element
        fn = np.sum(cm[i, :]) - tp
        
        # IoU calculation
        if tp + fp + fn == 0:
            class_iou[i] = 0
        else:
            class_iou[i] = tp / (tp + fp + fn)
    
    # Calculate mean IoU
    mean_iou = np.mean(class_iou)
    
    return class_iou, mean_iou

# Use your raw confusion matrix (before normalization)
class_iou, mean_iou = calculate_iou_from_cm(cm)  # 'cm' is your raw confusion matrix

print("Class IoU values:")
for i, class_name in enumerate(classes):
    print(f"{class_name}: {class_iou[i]:.4f}")
print(f"Mean IoU: {mean_iou:.4f}")

In [ ]:
#Metric Precision, Recall, F-score support
from sklearn.metrics import precision_recall_fscore_support

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

macro_01 = precision_recall_fscore_support(y_test, y_pred, average='macro')
weighted_01 = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(macro_01)
print(weighted_01)

In [ ]:
#Metric F1-score
from sklearn.metrics import f1_score

macro_02 = f1_score(y_test, y_pred, average='macro')
weighted_02 = f1_score(y_test, y_pred, average='weighted')

print(macro_02)
print(weighted_02)

In [ ]:
# Print the precision and recall, among other metrics
from sklearn import metrics
from tensorflow.compat.v1.keras.models import load_model

# model = load_model('D:/Wiltang_2022/Paper_Data_Codes/Saved_Model/LC_Multivariate_Wonosobo_BiGRU_exp_ver01_metricmeasurement.h5')
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)
# y_test = y_test.argmax(axis=1)

print(metrics.classification_report(y_test, y_pred, digits=4))

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

y_pred_train = model.predict(X_train)
y_pred_train = y_pred_train.argmax(axis=1)
y_train = y_train.argmax(axis=1)

JS_test = jaccard_score(y_test, y_pred, average='macro')
JS_train = jaccard_score(y_train, y_pred_train, average='macro')
print(JS_test)
print(JS_train)

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

y_pred_train = model.predict(X_train)
y_pred_train = y_pred_train.argmax(axis=1)
# y_train = y_train.argmax(axis=1)

JS_test = jaccard_score(y_test, y_pred, average=None)
JS_train = jaccard_score(y_train, y_pred_train, average=None)
print(JS_test)
print(JS_train)

In [ ]:
from sklearn.metrics import cohen_kappa_score

model_kappa = cohen_kappa_score(y_pred, y_test)
print(model_kappa)

In [ ]:
# data_test = {'ytest': y_test,
#         'ytest_pred': y_pred}

# data_train = {'ytest': y_train,
#         'ytest_pred': y_pred_train}

# df_test = pd.DataFrame(data_test)
# df_train = pd.DataFrame(data_train)

# df_test.to_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/df_test.csv')
# df_train.to_csv('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/df_train.csv')

In [ ]:
# from tensorflow.keras.utils import plot_model
# import pydot
# import pydotplus
# from pydotplus import graphviz

# plot_model(model, to_file='D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Saved_Model/model3_plot.png', 
#            show_shapes=True, show_layer_names=True)

In [ ]:
#https://doi.org/10.3390/agronomy11040654